In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# Load your trained model
model = load_model('sign_language_model.keras')  # or 'best_model.keras'

# Get class labels (update with your actual class names)
class_labels = ['A', 'B', 'C', 'D','DELETE', 'E' ,'F', 'G', 'H','I','J','K','L','M','N','O','P','Q','R','S','Space','T','U','V','W','X','Y','Z']  # Replace with your classes

def extract_landmarks(image):
    """Extract hand landmarks from image"""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    
    if results.multi_hand_landmarks:
        landmarks = []
        for hand_landmarks in results.multi_hand_landmarks:
            # Extract all 21 landmarks (x,y,z)
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])
            # Draw landmarks on image
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        # Pad with zeros if we only got one hand (for consistent input size)
        if len(landmarks) < 63:  # 21 landmarks * 3 values
            landmarks.extend([0] * (63 - len(landmarks)))
        return np.array(landmarks, dtype=np.float32)
    return None


def predict_sign(landmarks):
    """Predict sign from landmarks"""
    # Reshape for model input (1 sample with 63 features)
    landmarks = landmarks.reshape(1, -1)
    prediction = model.predict(landmarks, verbose=0)
    predicted_class = np.argmax(prediction)
    confidence = np.max(prediction)
    return class_labels[predicted_class], confidence

def main():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return
    
    print("Starting real-time prediction...")
    print("Press 'q' to quit")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Flip frame horizontally for mirror effect
        frame = cv2.flip(frame, 1)
        
        # Extract landmarks
        landmarks = extract_landmarks(frame)
        
        if landmarks is not None:
            # Make prediction
            sign, confidence = predict_sign(landmarks)
            
            # Display prediction
            cv2.putText(frame, f"Sign: {sign}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Confidence: {confidence:.2f}", (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Display instructions
        cv2.putText(frame, "Press 'q' to quit", (10, frame.shape[0] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # Show the frame
        cv2.imshow('Sign Language Prediction', frame)
        
        # Exit on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Starting real-time prediction...
Press 'q' to quit
